In [2]:
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import pre_processing as pp
import csv
import pandas as pd
import tensorflow as tf

# Dataset ting

In [17]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('data/6-gon-correct-procrustes.csv')

df = df.iloc[:, :14]
dataset = df.drop(columns=['target_edge_length'])

training = dataset.sample(frac=0.85)
testing = dataset.drop(training.index)

train_features = training.iloc[:, :12]
train_labels = training.iloc[:, -1:]

test_features = testing.iloc[:, :12]
test_labels = testing.iloc[:, -1:]

# Model ting

In [19]:
import tensorflow as tf
from tensorflow.keras import layers

model_path = "model/thesis-internal-node-count-6gon"
BATCH_SIZE = 512
EPOCHS = 5000
INITIAL_LEARNING_RATE = 1e-1

def model_setup():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(12,)),
        layers.BatchNormalization(),

        layers.Dense(24, activation='relu'),
        layers.BatchNormalization(),

        layers.Dense(24, activation='relu'),
        layers.BatchNormalization(),

        layers.Dense(1),
    ])
    return model
model = model_setup()
# model.summary()

decay_steps= 10_000
lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    INITIAL_LEARNING_RATE, 
    decay_steps, 
    1e-3
    )

model.compile(loss=tf.losses.MeanAbsoluteError(),
              optimizer=tf.optimizers.Adam(
              learning_rate=learning_rate
        ),
        )

checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=EPOCHS//10, min_delta=0.0001)

history = model.fit(train_features,
                    train_labels,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_split=0.18,
                    verbose=2,
                    callbacks=[checkpoint, early_stopping],
                    )

# ======================
#       EVALUATION
# ======================
train_acc = model.evaluate(
    train_features, train_labels, verbose=0)
test_acc = model.evaluate(
    test_features, test_labels, verbose=0)
print('Training data loss: %.3f, Test data loss: %.3f' %
        (train_acc, test_acc))      


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 12)                48        
_________________________________________________________________
dense (Dense)                (None, 24)                312       
_________________________________________________________________
batch_normalization_1 (Batch (None, 24)                96        
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
batch_normalization_2 (Batch (None, 24)                96        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 25        
Total params: 1,177
Trainable params: 1,057
Non-trainable params: 120
____________________________________________________

In [ ]:
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("MAE")